! git clone https://github.com/cynricfu/MAGNN.git

In [1]:
%%capture
! pip install transformers==3.3.1

In [2]:
%%capture
! pip install torch==1.3.0

In [3]:
%%capture
! pip install dgl-cu101==0.4.1

In [2]:
import pathlib
import numpy as np
import scipy.sparse
import scipy.io
import pandas as pd
import transformers

# from sklearn.feature_extraction.text import CountVectorizer

import networkx as nx

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os

if os.getcwd()=="/content/drive/My Drive/Colab Notebooks/MAGNN":
    True
else:
    os.chdir("/content/drive/My Drive/Colab Notebooks/MAGNN")

In [5]:
import utils.preprocess
from sklearn.model_selection import train_test_split

import time

import torch
import torch.nn.functional as F
import torch.sparse
import dgl

from utils.pytorchtools import EarlyStopping
from utils.data import load_IMDB_data
from utils.tools import evaluate_results_nc
from model import MAGNN_nc


In [6]:
# from transformers import DistilBertTokenizer, DistilBertModel
import torch

In [ ]:
from distillBERT import chunks, fetch_vectors

In [8]:
# save_prefix = 'data/preprocessed/IMDB_processed/'
num_ntypes = 3

# load raw data, delete movies with no actor or director
movies = pd.read_csv('./data/raw/IMDB/movie_metadata.csv', encoding='utf-8').dropna(
    axis=0, subset=['actor_1_name', 'director_name']).reset_index(drop=True)


In [9]:

# extract labels, and delete movies with unwanted genres
# 0 for action, 1 for comedy, 2 for drama, -1 for others
labels = np.zeros((len(movies)), dtype=int)
for movie_idx, genres in movies['genres'].iteritems():
    labels[movie_idx] = -1
    for genre in genres.split('|'):
        if genre == 'Action':
            labels[movie_idx] = 0
            break
        elif genre == 'Comedy':
            labels[movie_idx] = 1
            break
        elif genre == 'Drama':
            labels[movie_idx] = 2
            break


In [10]:
unwanted_idx = np.where(labels == -1)[0]

movies = movies.drop(unwanted_idx).reset_index(drop=True)

labels = np.delete(labels, unwanted_idx, 0)

In [11]:

# get director list and actor list

directors = list(set(movies['director_name'].dropna()))

directors.sort()


In [12]:

actors = list(set(movies['actor_1_name'].dropna().to_list() +
                  movies['actor_2_name'].dropna().to_list() +
                  movies['actor_3_name'].dropna().to_list()))

actors.sort()


In [13]:
# build the adjacency matrix for the graph consisting of movies, directors and actors
# 0 for movies, 1 for directors, 2 for actors
dim = len(movies) + len(directors) + len(actors)


In [14]:
type_mask = np.zeros((dim), dtype=int)

type_mask[len(movies):len(movies)+len(directors)] = 1

type_mask[len(movies)+len(directors):] = 2


In [15]:
adjM = np.zeros((dim, dim), dtype=int)
for movie_idx, row in movies.iterrows():
    if row['director_name'] in directors:
        director_idx = directors.index(row['director_name'])
        adjM[movie_idx, len(movies) + director_idx] = 1
        adjM[len(movies) + director_idx, movie_idx] = 1
    if row['actor_1_name'] in actors:
        actor_idx = actors.index(row['actor_1_name'])
        adjM[movie_idx, len(movies) + len(directors) + actor_idx] = 1
        adjM[len(movies) + len(directors) + actor_idx, movie_idx] = 1
    if row['actor_2_name'] in actors:
        actor_idx = actors.index(row['actor_2_name'])
        adjM[movie_idx, len(movies) + len(directors) + actor_idx] = 1
        adjM[len(movies) + len(directors) + actor_idx, movie_idx] = 1
    if row['actor_3_name'] in actors:
        actor_idx = actors.index(row['actor_3_name'])
        adjM[movie_idx, len(movies) + len(directors) + actor_idx] = 1
        adjM[len(movies) + len(directors) + actor_idx, movie_idx] = 1


In [16]:
# extract bag-of-word representations of plot keywords for each movie
# X is a sparse matrix
# vectorizer = CountVectorizer(min_df=2)
# movie_X = vectorizer.fit_transform(movies['plot_keywords'].fillna('').values)
keywords=movies.plot_keywords.fillna('').apply(lambda x: str(x).replace("|"," "))

In [ ]:
embedding_value=fetch_vectors(keywords)

In [ ]:
embedding_value=scipy.sparse.csr_matrix(embedding_value)

In [18]:
save_prefix='./data/preprocessed/IMDB_processed_bert/'

In [19]:


# assign features to directors and actors as the means of their associated movies' features
adjM_da2m = adjM[len(movies):, :len(movies)]

adjM_da2m_normalized = np.diag(1 / adjM_da2m.sum(axis=1)).dot(adjM_da2m)

director_actor_X =scipy.sparse.csr_matrix(adjM_da2m_normalized).dot(embedding_value_raw)


In [20]:
full_X = scipy.sparse.vstack([embedding_value_raw, director_actor_X])

In [21]:

expected_metapaths = [
    [(0, 1, 0), (0, 2, 0)],
    [(1, 0, 1), (1, 0, 2, 0, 1)],
    [(2, 0, 2), (2, 0, 1, 0, 2)]
]


# create the directories if they do not exist

for i in range(num_ntypes):
    pathlib.Path(save_prefix + '{}'.format(i)).mkdir(parents=True, exist_ok=True)



for i in range(num_ntypes):
    # get metapath based neighbor pairs
    neighbor_pairs = utils.preprocess.get_metapath_neighbor_pairs(adjM, type_mask, expected_metapaths[i])
    # construct and save metapath-based networks
    G_list = utils.preprocess.get_networkx_graph(neighbor_pairs, type_mask, i)
    
    # save data
    # networkx graph (metapath specific)
    for G, metapath in zip(G_list, expected_metapaths[i]):
        nx.write_adjlist(G, save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '.adjlist')
    # node indices of edge metapaths
    all_edge_metapath_idx_array = utils.preprocess.get_edge_metapath_idx_array(neighbor_pairs)
    for metapath, edge_metapath_idx_array in zip(expected_metapaths[i], all_edge_metapath_idx_array):
        np.save(save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '_idx.npy', edge_metapath_idx_array)

# save data
# all nodes adjacency matrix
scipy.sparse.save_npz(save_prefix + 'adjM.npz', scipy.sparse.csr_matrix(adjM))



(17446, 3)
(95102, 3)
(4278, 3)
(95102, 5)
(38476, 3)
(156928, 5)


In [22]:

# all nodes (movies, directors and actors) features
for i in range(num_ntypes):
    scipy.sparse.save_npz(save_prefix + 'features_{}.npz'.format(i), scipy.sparse.csr_matrix(full_X.todense()[np.where(type_mask == i)[0]]))


In [23]:
# all nodes (movies, directors and actors) type labels
np.save(save_prefix + 'node_types.npy', type_mask)


# movie genre labels
np.save(save_prefix + 'labels.npy', labels)


# movie train/validation/test splits
rand_seed = 1566911444

train_idx, val_idx = train_test_split(np.arange(len(labels)), test_size=400, random_state=rand_seed)
train_idx, test_idx = train_test_split(train_idx, test_size=3478, random_state=rand_seed)

train_idx.sort()
val_idx.sort()
test_idx.sort()


np.savez(save_prefix + 'train_val_test_idx.npz',
         val_idx=val_idx,
         train_idx=train_idx,
         test_idx=test_idx)

# Run Model

In [24]:

# Params
out_dim = 3
dropout_rate = 0.5
lr = 0.005
weight_decay = 0.001
etypes_lists = [[[0, 1], [2, 3]],
                [[1, 0], [1, 2, 3, 0]],
                [[3, 2], [3, 0, 1, 2]]]


In [25]:

def run_model_IMDB(feats_type, num_layers, hidden_dim, num_heads, attn_vec_dim, rnn_type,
                   num_epochs, patience, repeat, save_postfix):
    nx_G_lists, edge_metapath_indices_lists, features_list, adjM, type_mask, labels, train_val_test_idx = load_IMDB_data()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    features_list = [torch.FloatTensor(features.todense()).to(device) for features in features_list]

    if feats_type == 0:
        in_dims = [features.shape[1] for features in features_list]
    
    elif feats_type == 1:
        in_dims = [features_list[0].shape[1]] + [10] * (len(features_list) - 1)
        for i in range(1, len(features_list)):
            features_list[i] = torch.zeros((features_list[i].shape[0], 10)).to(device)
    
    elif feats_type == 2:
        in_dims = [features.shape[0] for features in features_list]
        in_dims[0] = features_list[0].shape[1]
        for i in range(1, len(features_list)):
            dim = features_list[i].shape[0]
            indices = np.vstack((np.arange(dim), np.arange(dim)))
            indices = torch.LongTensor(indices)
            values = torch.FloatTensor(np.ones(dim))
            features_list[i] = torch.sparse.FloatTensor(indices, values, torch.Size([dim, dim])).to(device)
    
    elif feats_type == 3:
        in_dims = [features.shape[0] for features in features_list]
        for i in range(len(features_list)):
            dim = features_list[i].shape[0]
            indices = np.vstack((np.arange(dim), np.arange(dim)))
            indices = torch.LongTensor(indices)
            values = torch.FloatTensor(np.ones(dim))
            features_list[i] = torch.sparse.FloatTensor(indices, values, torch.Size([dim, dim])).to(device)
    
    edge_metapath_indices_lists = [[torch.LongTensor(indices).to(device) for indices in indices_list] for indices_list in
                                   edge_metapath_indices_lists]
    
    labels = torch.LongTensor(labels).to(device)
    
    g_lists = []
    
    for nx_G_list in nx_G_lists:
        g_lists.append([])
        for nx_G in nx_G_list:
            g = dgl.DGLGraph(multigraph=True)
            g.add_nodes(nx_G.number_of_nodes())
            g.add_edges(*list(zip(*sorted(map(lambda tup: (int(tup[0]), int(tup[1])), nx_G.edges())))))
            g_lists[-1].append(g)
    

    train_idx = train_val_test_idx['train_idx']
    val_idx = train_val_test_idx['val_idx']
    test_idx = train_val_test_idx['test_idx']


    svm_macro_f1_lists = []
    svm_micro_f1_lists = []

    nmi_mean_list = []
    nmi_std_list = []
    
    ari_mean_list = []
    ari_std_list = []
    for _ in range(repeat):
        net = MAGNN_nc(num_layers, [2, 2, 2], 4, etypes_lists, in_dims, hidden_dim, out_dim, num_heads, attn_vec_dim,
                       rnn_type, dropout_rate)
        net.to(device)
        optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
        target_node_indices = np.where(type_mask == 0)[0]

        # training loop
        net.train()
        early_stopping = EarlyStopping(patience=patience, verbose=True, save_path='checkpoint_bert/checkpoint_{}.pt'.format(save_postfix))
        dur1 = []
        dur2 = []
        dur3 = []
        for epoch in range(num_epochs):
            t0 = time.time()

            # training forward
            net.train()
            logits, embeddings = net((g_lists, features_list, type_mask, edge_metapath_indices_lists), target_node_indices)
            logp = F.log_softmax(logits, 1)
            train_loss = F.nll_loss(logp[train_idx], labels[train_idx])

            t1 = time.time()
            dur1.append(t1 - t0)

            # autograd
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            t2 = time.time()
            dur2.append(t2 - t1)

            # validation forward
            net.eval()
            with torch.no_grad():
                logits, embeddings = net((g_lists, features_list, type_mask, edge_metapath_indices_lists), target_node_indices)
                logp = F.log_softmax(logits, 1)
                val_loss = F.nll_loss(logp[val_idx], labels[val_idx])

            t3 = time.time()
            dur3.append(t3 - t2)

            # print info
            print(
                "Epoch {:05d} | Train_Loss {:.4f} | Val_Loss {:.4f} | Time1(s) {:.4f} | Time2(s) {:.4f} | Time3(s) {:.4f}".format(
                    epoch, train_loss.item(), val_loss.item(), np.mean(dur1), np.mean(dur2), np.mean(dur3)))
            # early stopping
            early_stopping(val_loss, net)
            if early_stopping.early_stop:
                print('Early stopping!')
                break


        # testing with evaluate_results_nc
        net.load_state_dict(torch.load('checkpoint_bert/checkpoint_{}.pt'.format(save_postfix)))
        net.eval()
        with torch.no_grad():
            logits, embeddings = net((g_lists, features_list, type_mask, edge_metapath_indices_lists), target_node_indices)
            svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = evaluate_results_nc(
                embeddings[test_idx].cpu().numpy(), labels[test_idx].cpu().numpy(), num_classes=out_dim)
        svm_macro_f1_lists.append(svm_macro_f1_list)
        svm_micro_f1_lists.append(svm_micro_f1_list)
        nmi_mean_list.append(nmi_mean)
        nmi_std_list.append(nmi_std)
        ari_mean_list.append(ari_mean)
        ari_std_list.append(ari_std)

    # print out a summary of the evaluations
    svm_macro_f1_lists = np.transpose(np.array(svm_macro_f1_lists), (1, 0, 2))
    svm_micro_f1_lists = np.transpose(np.array(svm_micro_f1_lists), (1, 0, 2))
    nmi_mean_list = np.array(nmi_mean_list)
    nmi_std_list = np.array(nmi_std_list)
    ari_mean_list = np.array(ari_mean_list)
    ari_std_list = np.array(ari_std_list)
    print('----------------------------------------------------------------')
    print('SVM tests summary')
    print('Macro-F1: ' + ', '.join(['{:.6f}~{:.6f} ({:.1f})'.format(
        macro_f1[:, 0].mean(), macro_f1[:, 1].mean(), train_size) for macro_f1, train_size in
        zip(svm_macro_f1_lists, [0.8, 0.6, 0.4, 0.2])]))
    print('Micro-F1: ' + ', '.join(['{:.6f}~{:.6f} ({:.1f})'.format(
        micro_f1[:, 0].mean(), micro_f1[:, 1].mean(), train_size) for micro_f1, train_size in
        zip(svm_micro_f1_lists, [0.8, 0.6, 0.4, 0.2])]))
    print('K-means tests summary')
    print('NMI: {:.6f}~{:.6f}'.format(nmi_mean_list.mean(), nmi_std_list.mean()))
    print('ARI: {:.6f}~{:.6f}'.format(ari_mean_list.mean(), ari_std_list.mean()))



In [26]:
feats_type=2
layers=2
hidden_dim=64
num_heads=8
attn_vec_dim=128
rnn_type='RotatE0'
epoch=100
patience=10
repeat=1
save_postfix='IMDB'

In [27]:
run_model_IMDB(feats_type, layers, hidden_dim, num_heads, attn_vec_dim, rnn_type,
                epoch, patience, repeat, save_postfix)

Epoch 00000 | Train_Loss 1.0962 | Val_Loss 1.0326 | Time1(s) 0.2773 | Time2(s) 0.4518 | Time3(s) 0.1180
Validation loss decreased (inf --> 1.032593).  Saving model ...
Epoch 00001 | Train_Loss 0.9931 | Val_Loss 0.9871 | Time1(s) 0.2009 | Time2(s) 0.4452 | Time3(s) 0.1166
Validation loss decreased (1.032593 --> 0.987097).  Saving model ...
Epoch 00002 | Train_Loss 0.8667 | Val_Loss 0.9185 | Time1(s) 0.1749 | Time2(s) 0.4423 | Time3(s) 0.1168
Validation loss decreased (0.987097 --> 0.918530).  Saving model ...
Epoch 00003 | Train_Loss 0.6647 | Val_Loss 0.8515 | Time1(s) 0.1622 | Time2(s) 0.4423 | Time3(s) 0.1168
Validation loss decreased (0.918530 --> 0.851500).  Saving model ...
Epoch 00004 | Train_Loss 0.4202 | Val_Loss 0.8634 | Time1(s) 0.1548 | Time2(s) 0.4444 | Time3(s) 0.1182
EarlyStopping counter: 1 out of 10
Epoch 00005 | Train_Loss 0.2706 | Val_Loss 0.9565 | Time1(s) 0.1513 | Time2(s) 0.4479 | Time3(s) 0.1175
EarlyStopping counter: 2 out of 10
Epoch 00006 | Train_Loss 0.1456 | V